# Homework 1 - Problem 2

Reload data from the open data portal

In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import matplotlib.pyplot as plt

client = Socrata("data.cityofchicago.org", None)
results = client.get("6zsd-86xi", where="year=2017 OR year=2018",limit=1000000)
results_df = pd.DataFrame.from_records(results)

Note: source consulted for geomerge https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

Convert into geodataframe to be able to merge to census API
First, load necessary packages

In [2]:
import geopandas
import shapefile as shp
from shapely.geometry import Point

Create coordinate object to use as shapely object

In [61]:
results_df['latitude'] = pd.to_numeric(results_df['latitude'])
results_df['longitude'] = pd.to_numeric(results_df['longitude'])
results_df.dropna(axis=0, how='any',subset=['latitude','longitude'], inplace=True)
results_df['Coordinates'] = list(zip(results_df.longitude, results_df.latitude))
results_df['Coordinates'] = results_df['Coordinates'].apply(Point)
gdf = geopandas.GeoDataFrame(results_df, geometry='Coordinates')

Now, load ACS data via census API 
(source consulted - documentation)

In [45]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)
import requests

In [44]:
#Make the API requests - variables' names updated below to reflect pulls; attempting to look at income, race, and employment levels
request_obj = requests.get('https://api.census.gov/data/2017/acs/acs5/?get=NAME,B19013_001E,B03002_003E,B03002_001E,B23025_005E,B23025_003E&for=block%20group:*&in=state:17%20county:031')
json2 = request_obj.json()
df_2 = pd.DataFrame(json2)
df_2.columns = df_2.iloc[0]
df_2 = df_2.drop(df_2.index[0])
df_2 = df_2.rename(columns={'B19013_001E': 'median_income', 'B23025_005E': 'unemployed','B23025_003E': 'labor_force', 'B03002_003E': 'non_hisp_white','B03002_001E': 'total'})
df_2['unemployed'] = pd.to_numeric(df_2['unemployed'])
df_2['labor_force'] = pd.to_numeric(df_2['labor_force'])
df_2['percent_unemp'] = df_2['unemployed']/df_2['labor_force']
df_2['non_hisp_white'] = pd.to_numeric(df_2['non_hisp_white'])
df_2['total'] = pd.to_numeric(df_2['total'])
df_2['percent_white'] = df_2['non_hisp_white']/df_2['total']
df_2.head()

,NAME,median_income,non_hisp_white,total,unemployed,labor_force,state,county,tract,block group,percent_unemp,percent_white
1,"Block Group 1, Census Tract 2519, Cook County,...",21500,11,600,67,209,17,031,251900,1,3.21e-01,0.02
2,"Block Group 2, Census Tract 2519, Cook County,...",32143,78,1546,215,565,17,031,251900,2,3.81e-01,0.05
3,"Block Group 1, Census Tract 8211.01, Cook Coun...",57000,712,1341,43,597,17,031,821101,1,7.20e-02,0.53
4,"Block Group 3, Census Tract 8211.01, Cook Coun...",65699,778,1356,14,461,17,031,821101,3,3.04e-02,0.57
5,"Block Group 2, Census Tract 8211.01, Cook Coun...",97656,599,616,4,405,17,031,821101,2,9.88e-03,0.97


In [6]:
#Create shapefile to use with census data to get locations
shp_path = "./tl_2018_17_bg.shp"
location_match = geopandas.read_file(shp_path)

In [50]:
new_df = df_2.merge(location_match,how='left',left_on=['tract','block group'],right_on=['TRACTCE','BLKGRPCE'])
new_df.head()

,NAME,median_income,non_hisp_white,total,unemployed,labor_force,state,county,tract,block group,...,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,"Block Group 1, Census Tract 2519, Cook County,...",21500,11,600,67,209,17,031,251900,1,...,1,170312519001,Block Group 1,G5030,S,117328,0,+41.8838925,-087.7593102,"POLYGON ((-87.76035399999999 41.887358, -87.76..."
1,"Block Group 2, Census Tract 2519, Cook County,...",32143,78,1546,215,565,17,031,251900,2,...,2,170312519002,Block Group 2,G5030,S,207721,0,+41.8837977,-087.7568297,"POLYGON ((-87.75899799999999 41.887377, -87.75..."
2,"Block Group 1, Census Tract 8211.01, Cook Coun...",57000,712,1341,43,597,17,031,821101,1,...,1,170318211011,Block Group 1,G5030,S,573854,0,+41.7407355,-087.7651488,"POLYGON ((-87.773765 41.737974, -87.7730529999..."
3,"Block Group 3, Census Tract 8211.01, Cook Coun...",65699,778,1356,14,461,17,031,821101,3,...,3,170318211013,Block Group 3,G5030,S,409521,0,+41.7360810,-087.7566885,"POLYGON ((-87.762828 41.737799, -87.762585 41...."
4,"Block Group 2, Census Tract 8211.01, Cook Coun...",97656,599,616,4,405,17,031,821101,2,...,2,170318211012,Block Group 2,G5030,S,328063,0,+41.7397534,-087.7556097,"POLYGON ((-87.76053499999999 41.741493, -87.76..."


In [54]:
census_gdf = geopandas.GeoDataFrame(new_df, geometry='geometry')

In [58]:
#Grab only the columns that we want
census_gdf = census_gdf[['NAME','median_income','tract', 'block group',
       'percent_unemp', 'percent_white', 'geometry']]
census_gdf.head()

,NAME,median_income,tract,block group,percent_unemp,percent_white,geometry
0,"Block Group 1, Census Tract 2519, Cook County,...",21500,251900,1,3.21e-01,0.02,"POLYGON ((-87.76035399999999 41.887358, -87.76..."
1,"Block Group 2, Census Tract 2519, Cook County,...",32143,251900,2,3.81e-01,0.05,"POLYGON ((-87.75899799999999 41.887377, -87.75..."
2,"Block Group 1, Census Tract 8211.01, Cook Coun...",57000,821101,1,7.20e-02,0.53,"POLYGON ((-87.773765 41.737974, -87.7730529999..."
3,"Block Group 3, Census Tract 8211.01, Cook Coun...",65699,821101,3,3.04e-02,0.57,"POLYGON ((-87.762828 41.737799, -87.762585 41...."
4,"Block Group 2, Census Tract 8211.01, Cook Coun...",97656,821101,2,9.88e-03,0.97,"POLYGON ((-87.76053499999999 41.741493, -87.76..."


In [62]:
census_merge = geopandas.sjoin(gdf, census_gdf, how="inner", op='intersects')

In [63]:
census_merge.head()

,arrest,beat,block,case_number,community_area,date,description,district,domestic,fbi_code,...,y_coordinate,year,Coordinates,index_right,NAME,median_income,tract,block group,percent_unemp,percent_white
0,True,1034,026XX S CALIFORNIA BLVD,JA529032,30,2017-11-28T21:43:00.000,VIOLENT OFFENDER: ANNUAL REGISTRATION,010,False,26,...,1886310,2017,POINT (-87.69463767799999 41.843778126),2867,"Block Group 1, Census Tract 8435, Cook County,...",25714,843500,1,0.08,0.09
570,False,1032,026XX S KEDZIE AVE,JB116615,30,2018-01-14T18:00:00.000,OVER $500,010,False,06,...,1886230,2018,POINT (-87.70501439899999 41.843615819),2867,"Block Group 1, Census Tract 8435, Cook County,...",25714,843500,1,0.08,0.09
6076,False,1033,026XX S CALIFORNIA AVE,JA107780,30,2017-01-07T21:15:00.000,SEX OFFENDER: FAIL TO REGISTER,010,False,26,...,1886128,2017,POINT (-87.695171094 41.843281635),2867,"Block Group 1, Census Tract 8435, Cook County,...",25714,843500,1,0.08,0.09
7526,False,1033,026XX S CALIFORNIA AVE,JA110662,30,2017-01-10T13:00:00.000,OVER $500,010,False,06,...,1886449,2017,POINT (-87.695199039 41.844162699),2867,"Block Group 1, Census Tract 8435, Cook County,...",25714,843500,1,0.08,0.09
7705,False,1033,031XX W 27TH ST,JA118537,30,2017-01-15T16:00:00.000,TO VEHICLE,010,False,14,...,1885900,2017,POINT (-87.70424527 41.842706001),2867,"Block Group 1, Census Tract 8435, Cook County,...",25714,843500,1,0.08,0.09
